In [1]:
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFilter, ImageOps
import random
import os
import auto_maskmix_util
import matplotlib.pyplot as plt

In [2]:
save_dir = "/home/tamaru/scene_categorize/main/data/augmented_image"

In [3]:
mask_cube = pd.read_csv("/home/tamaru/scene_categorize/main/data/mask_cube.csv", names=["path", "data", "mask_cube"])
mix_img = pd.read_csv("/home/tamaru/scene_categorize/main/data/mix_img.csv", names=["path", "data", "mix_img"])

In [4]:
mask_cube = mask_cube.drop(["data", "mask_cube"], axis=1)
mix_img = mix_img.drop(["data", "mix_img"], axis=1)

In [5]:
mask_cube["head"] = mask_cube.path.str.split("/", expand=True)[8]

In [6]:
bg_mask = mask_cube[mask_cube["head"].str.contains("mask")]

In [7]:
bg_mask = bg_mask.sort_values("head")
bg_mask = bg_mask.reset_index(drop=True)
bg_mask

path  \
0   /home/tamaru/scene_categorize/main/data/mask_c...   
1   /home/tamaru/scene_categorize/main/data/mask_c...   
2   /home/tamaru/scene_categorize/main/data/mask_c...   
3   /home/tamaru/scene_categorize/main/data/mask_c...   
4   /home/tamaru/scene_categorize/main/data/mask_c...   
5   /home/tamaru/scene_categorize/main/data/mask_c...   
6   /home/tamaru/scene_categorize/main/data/mask_c...   
7   /home/tamaru/scene_categorize/main/data/mask_c...   
8   /home/tamaru/scene_categorize/main/data/mask_c...   
9   /home/tamaru/scene_categorize/main/data/mask_c...   
10  /home/tamaru/scene_categorize/main/data/mask_c...   
11  /home/tamaru/scene_categorize/main/data/mask_c...   
12  /home/tamaru/scene_categorize/main/data/mask_c...   
13  /home/tamaru/scene_categorize/main/data/mask_c...   
14  /home/tamaru/scene_categorize/main/data/mask_c...   
15  /home/tamaru/scene_categorize/main/data/mask_c...   

                                head  
0   lab_bs_cnt_0_img_000030_mask.png  
1   lab_corner_0_img_000100_mask.png  
2   lab_corner_1_img_000220_mask.png  
3     lab_desk_0_img_000030_mask.png  
4    lab_desk_11_img_000280_mask.png  
5    lab_desk_14_img_000310_mask.png  
6    lab_desk_16_img_000120_mask.png  
7    lab_desk_16_img_000270_mask.png  
8    lab_desk_17_img_000040_mask.png  
9     lab_desk_6_img_000160_mask.png  
10    lab_desk_8_img_000120_mask.png  
11    lab_desk_9_img_000100_mask.png  
12   lab_table_0_img_000030_mask.png  
13  lab_table_10_img_000240_mask.png  
14  lab_table_21_img_000130_mask.png  
15   lab_table_2_img_000140_mask.png

In [8]:
background = mask_cube[mask_cube["head"].str.contains("mask") == False]

In [9]:
background = background.sort_values("head")
background = background.reset_index(drop=True)
background

path  \
0   /home/tamaru/scene_categorize/main/data/mask_c...   
1   /home/tamaru/scene_categorize/main/data/mask_c...   
2   /home/tamaru/scene_categorize/main/data/mask_c...   
3   /home/tamaru/scene_categorize/main/data/mask_c...   
4   /home/tamaru/scene_categorize/main/data/mask_c...   
5   /home/tamaru/scene_categorize/main/data/mask_c...   
6   /home/tamaru/scene_categorize/main/data/mask_c...   
7   /home/tamaru/scene_categorize/main/data/mask_c...   
8   /home/tamaru/scene_categorize/main/data/mask_c...   
9   /home/tamaru/scene_categorize/main/data/mask_c...   
10  /home/tamaru/scene_categorize/main/data/mask_c...   
11  /home/tamaru/scene_categorize/main/data/mask_c...   
12  /home/tamaru/scene_categorize/main/data/mask_c...   
13  /home/tamaru/scene_categorize/main/data/mask_c...   
14  /home/tamaru/scene_categorize/main/data/mask_c...   
15  /home/tamaru/scene_categorize/main/data/mask_c...   

                           head  
0   lab_bs_cnt_0_img_000030.png  
1   lab_corner_0_img_000100.png  
2   lab_corner_1_img_000220.png  
3     lab_desk_0_img_000030.png  
4    lab_desk_11_img_000280.png  
5    lab_desk_14_img_000310.png  
6    lab_desk_16_img_000120.png  
7    lab_desk_16_img_000270.png  
8    lab_desk_17_img_000040.png  
9     lab_desk_6_img_000160.png  
10    lab_desk_8_img_000120.png  
11    lab_desk_9_img_000100.png  
12   lab_table_0_img_000030.png  
13  lab_table_10_img_000240.png  
14  lab_table_21_img_000130.png  
15   lab_table_2_img_000140.png

In [22]:
mix_img = mix_img.sort_values("path")
mix_img = mix_img.reset_index(drop=True)
print(mix_img.path[0])

/home/tamaru/scene_categorize/main/data/mix_img/bag.jpg


In [23]:
fg_mask = pd.read_csv("/home/tamaru/scene_categorize/main/data/mask.csv", names=["path", "data", "mix_img"])
fg_mask = fg_mask.drop(["data", "mix_img"], axis=1)
fg_mask = fg_mask.sort_values("path")
fg_mask = fg_mask.reset_index(drop=True)
print(fg_mask.path[0])

/home/tamaru/scene_categorize/main/data/mask/bag.png


In [33]:
def auto_maskmix(FG_IMG, BG_IMG, SPEC_BG_IMG, fg_mask):
    foreground = Image.open(FG_IMG)
    background = Image.open(BG_IMG)
    foreground = foreground.resize((256,256))
    background = background.resize((256,256))
    fg_mask = Image.open(fg_mask)
#     fg_mask = ImageOps.invert(fg_mask)
    mask_im = fg_mask.resize(foreground.size).convert("L")
    
    coord = auto_maskmix_util.foreground_random_sampling(BG_IMG, SPEC_BG_IMG)
    
    back_im = background.copy()
    x, y = coord
    rate = np.random.rand()*0.5 + 0.5
    rotate = random.randint(0,360)
    foreground_ = foreground.resize((int(foreground.width*rate), int(foreground.height*rate))).rotate(rotate)
    mask_im_ = mask_im.resize((int(mask_im.width*rate), int(mask_im.height*rate))).rotate(rotate)
    #前景画像の配置位置の座標は前景画像の左上なので，中心に平行移動する->未完成
    back_im.paste(foreground_, (int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2)), mask_im_)
    print("Foreground center coordinate", int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2))
    print("Foreground Rate:", rate)
    print("Foreground Size", foreground_.size[1])
    return back_im
    

In [34]:
def auto_maskmix_2all(background, bg_mask, mix_img, fg_mask):
    bg_col_name = background.columns.values
    bg_mask_col_name = bg_mask.columns.values
    mix_col_name = mix_img.columns.values
    background_np = np.array(background)
    bg_mask_np = np.array(bg_mask)
    mix_img_np = np.array(mix_img)
    fg_mask_np = np.array(fg_mask)
    fg_mask_col_name = fg_mask.columns.values
    i = 0
    while(background_np[i][bg_col_name.tolist().index("path")] is not None):
        rand_mix = random.randint(0,len(mix_img_np)-1)
        print("Foreground:", rand_mix)
        back_im = auto_maskmix(mix_img_np[rand_mix][0], background_np[i][bg_col_name.tolist().index("path")], bg_mask_np[i][bg_mask_col_name.tolist().index("path")], fg_mask_np[rand_mix][0])
        back_im.save(save_dir+"/"+background_np[i][bg_col_name.tolist().index("head")])
        i += 1
        print("------------")
    

In [35]:
auto_maskmix_2all(background,bg_mask,mix_img, fg_mask)

Foreground: 10
Foreground center coordinate 82 42
Foreground Rate: 0.7324282738333094
Foreground Size 187
------------
Foreground: 0
Foreground center coordinate 81 -29
Foreground Rate: 0.8326966144899851
Foreground Size 213
------------
Foreground: 8
Foreground center coordinate -16 29
Foreground Rate: 0.8595949542285146
Foreground Size 220
------------
Foreground: 4
Foreground center coordinate 29 43
Foreground Rate: 0.7976750269272932
Foreground Size 204
------------
Foreground: 10
Foreground center coordinate 45 83
Foreground Rate: 0.6257251260133916
Foreground Size 160
------------
Foreground: 5
Foreground center coordinate 163 83
Foreground Rate: 0.5900670159524155
Foreground Size 151
------------
Foreground: 3
Foreground center coordinate 35 -12
Foreground Rate: 0.9603137736131119
Foreground Size 245
------------
Foreground: 4
Foreground center coordinate -2 -46
Foreground Rate: 0.905285172248008
Foreground Size 231
------------
Foreground: 5
Foreground center coordinate 143 108

IndexError: index 16 is out of bounds for axis 0 with size 16

mix_img_ = pd.read_csv("/home/tamaru/scene_categorize/main/data/mix_img.csv", names=["path", "data", "mix_img"])
mix_img_ = mix_img_.drop(["data", "mix_img"], axis=1)

def mask_with_grabcut(FG_IMG):
    bgr = cv2.imread(FG_IMG)
    bgr = cv2.cvtColor(bgr, cv2.COLOR_RGB2BGR)
    h, w = bgr.shape[:2]
    mask = np.zeros((h,w), dtype = np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect=(1,1,w,h)
    cv2.grabCut(bgr, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    bgr2 = bgr*mask2[:,:,np.newaxis]
    bgr2 = cv2.resize(bgr2, (256,256))
    # 輪郭抽出する。
    contours, _ = cv2.findContours(bgr2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 検出された輪郭内部を255で塗りつぶす。
    mask = np.zeros_like(bgr2)
    cv2.drawContours(mask, contours, -1, color=(255,255,255), thickness=-1)

    # 以上の手順で前景画像のうち、合成する画素を255としたマスク画像が作成できた。
    mask = cv2.resize(mask, (256,256))
    return mask


mix_img_np = np.array(mix_img_)
for row in mix_img_np: 
    print(row[0])
    print(row[0].split("/")[-1])
    mask = mask_with_grabcut(row[0])
    cv2.imwrite("/home/tamaru/scene_categorize/main/data/mask/"+row[0].split("/")[-1],mask)

mix_img_np = np.array(mix_img)
for row in mix_img_np: 
    print(row[0])
    print(row[0].split("/")[-1])
    image = cv2.imread(row[0], cv2.IMREAD_GRAYSCALE)
    # 2値化する。
    _, binary = cv2.threshold(image, 10, 255, cv2.THRESH_TOZERO)
    # 輪郭抽出する。
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 検出された輪郭内部を255で塗りつぶす。
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, color=(255,255,255), thickness=-1)
    mask =cv2.resize(mask, (256,256))
    cv2.imwrite("/home/tamaru/scene_categorize/main/data/mask/"+row[0].split("/")[-1],mask)